In [17]:
from rllab.algos.trpo import TRPO
from rllab.baselines.linear_feature_baseline import LinearFeatureBaseline
from rllab.envs.gym_env import GymEnv
from rllab.envs.normalized_env import normalize
from rllab.misc.instrument import run_experiment_lite
from rllab.policies.categorical_mlp_policy import CategoricalMLPPolicy
from rllab.policies.gaussian_mlp_policy import GaussianMLPPolicy
from rllab.policies.deterministic_mlp_policy import DeterministicMLPPolicy
from rllab.sampler.utils import collect_data

import gym
import numpy as np
import GPy
import GPyOpt

import matplotlib.pyplot as plt
%matplotlib inline

import matplotlib
import pandas as pd


import plotly
plotly.offline.init_notebook_mode() 
import plotly.plotly as py
import plotly.graph_objs as go

import numpy as np
import pandas as pd
import copy

In [3]:
path = collect_data(env, policy, max_path_length=env.horizon,
                       animated=False, speedup=1)
path

{'actions': array([[-0.88214474],
        [ 0.64992475],
        [-0.79706869],
        [-0.52112626]]), 'agent_infos': {'log_std': array([[-0.01014289],
         [-0.01014289],
         [-0.01014289],
         [-0.01014289]]), 'mean': array([[-0.05223089],
         [-0.57316388],
         [ 0.09151939],
         [-0.37779809]])}, 'env_infos': {}, 'observations': array([[  3.66252839e-03,  -6.19823099e-03,  -5.46348947e-03,
           2.72956905e-04],
        [ -1.59790954e-02,   3.92344607e-02,  -9.73468392e-01,
           2.24637670e+00],
        [ -4.03183671e-02,   9.33968083e-02,  -2.48127747e-01,
           5.01191595e-01],
        [ -6.78634092e-02,   1.55706686e-01,  -1.12525038e+00,
           2.58951268e+00],
        [ -1.23964438e-01,   2.84964846e-01,  -1.67583448e+00,
           3.86388075e+00]]), 'rewards': array([ 1.,  1.,  1.,  1.])}

In [4]:
states = path['observations']

In [5]:
states

array([[  3.66252839e-03,  -6.19823099e-03,  -5.46348947e-03,
          2.72956905e-04],
       [ -1.59790954e-02,   3.92344607e-02,  -9.73468392e-01,
          2.24637670e+00],
       [ -4.03183671e-02,   9.33968083e-02,  -2.48127747e-01,
          5.01191595e-01],
       [ -6.78634092e-02,   1.55706686e-01,  -1.12525038e+00,
          2.58951268e+00],
       [ -1.23964438e-01,   2.84964846e-01,  -1.67583448e+00,
          3.86388075e+00]])

In [6]:
actions = path['actions']

In [7]:
actions

array([[-0.88214474],
       [ 0.64992475],
       [-0.79706869],
       [-0.52112626]])

In [18]:
env2 = copy.deepcopy(env)

2017-09-05 17:09:06.193096 EDT | Warning: skipping Gym environment monitoring since snapshot_dir not configured.


[2017-09-05 17:09:06,195] Making new env: InvertedPendulum-v1


2017-09-05 17:09:06.203899 EDT | observation space: Box(4,)
2017-09-05 17:09:06.205310 EDT | action space: Box(1,)


In [23]:
env2.wrapped_env.env.env.model.body_mass

array([[ 0.        ],
       [ 9.42477796],
       [ 4.8953899 ]])

In [24]:
mb = env2.wrapped_env.env.env.model.body_mass
mb = np.array(mb)
mb[2,0] = mb[1,0]*1.4
env2.wrapped_env.env.env.model.body_mass = mb

In [31]:
env2.reset()

array([ 0.00214859, -0.00332162,  0.0005366 , -0.00429929])

In [32]:
env2.wrapped_env.env.env.set_state(states[0,:2],states[0,2:])

In [33]:
env2.wrapped_env.env.env.state_vector()

array([ 0.00366253, -0.00619823, -0.00546349,  0.00027296])

In [34]:
for i in range(0,4):
    next_o, r, d, env_info = env2.step(actions[i,:])
    print (actions[i,:],next_o)

[-0.88214474] [-0.01541496  0.04817815 -0.94581172  2.70764791]
[ 0.64992475] [-0.03938508  0.11694199 -0.26315033  0.78518592]
[-0.79706869] [-0.06735946  0.20071943 -1.1273773   3.37779775]
[-0.52112626] [-0.12294815  0.36873727 -1.63655418  4.99116682]


In [14]:
for i in range(0,4):
    next_o, r, d, env_info = env2.step(actions[i,:])
    print (actions[i,:],next_o)

[-0.88214474] [-0.0159791   0.03923446 -0.97346839  2.2463767 ]
[ 0.64992475] [-0.04031837  0.09339681 -0.24812775  0.5011916 ]
[-0.79706869] [-0.06786341  0.15570669 -1.12525038  2.58951268]
[-0.52112626] [-0.12396444  0.28496485 -1.67583448  3.86388075]


In [17]:
d

True

In [55]:
agent_info

{'log_std': array([-0.01957605]), 'mean': array([ 0.00387783])}

In [56]:
best_action

array([-1.30719061])

In [2]:
env = normalize(GymEnv("InvertedPendulum-v1"))

policy = GaussianMLPPolicy(
    env_spec=env.spec,
    # The neural network policy should have two hidden layers, each with 32 hidden units.
    hidden_sizes=(32, 32)
)

baseline = LinearFeatureBaseline(env_spec=env.spec)

algo = TRPO(
    env=env,
    policy=policy,
    baseline=baseline,
    batch_size=1,
    whole_paths=True,
    max_path_length=100,
    n_itr=1,
    discount=0.99,
    step_size=0.01,
)
algo.train()

2017-09-05 17:05:34.523442 EDT | Warning: skipping Gym environment monitoring since snapshot_dir not configured.


[2017-09-05 17:05:34,525] Making new env: InvertedPendulum-v1


2017-09-05 17:05:34.809888 EDT | observation space: Box(4,)
2017-09-05 17:05:34.811735 EDT | action space: Box(1,)
2017-09-05 17:05:35.646010 EDT | Populating workers...
2017-09-05 17:05:35.647389 EDT | Populated


0%                          100%
[##############################] | ETA: 00:00:00

2017-09-05 17:05:35.743083 EDT | itr #0 | fitting baseline...
2017-09-05 17:05:35.756271 EDT | itr #0 | fitted
=: Compiling function f_loss



Total time elapsed: 00:00:00


done in 0.325 seconds
=: Compiling function constraint
done in 0.158 seconds
2017-09-05 17:05:36.242084 EDT | itr #0 | computing loss before
2017-09-05 17:05:36.242891 EDT | itr #0 | performing update
2017-09-05 17:05:36.243553 EDT | itr #0 | computing descent direction
=: Compiling function f_grad
done in 0.415 seconds
=: Compiling function f_Hx_plain
done in 1.133 seconds
2017-09-05 17:05:37.881658 EDT | itr #0 | descent direction computed
=: Compiling function f_loss_constraint
done in 0.263 seconds
2017-09-05 17:05:38.148457 EDT | itr #0 | backtrack iters: 4
2017-09-05 17:05:38.149175 EDT | itr #0 | computing loss after
2017-09-05 17:05:38.149839 EDT | itr #0 | optimization finished
2017-09-05 17:05:38.150799 EDT | itr #0 | saving snapshot...
2017-09-05 17:05:38.151467 EDT | itr #0 | saved
2017-09-05 17:05:38.152583 EDT | -----------------------  ------------
2017-09-05 17:05:38.153241 EDT | Iteration                 0
2017-09-05 17:05:38.153839 EDT | AverageDiscountedReturn   4.90

In [8]:
state_history

NameError: name 'state_history' is not defined

In [53]:
env._state

[222,
 array([ 0.00018833, -0.00534661, -0.00188312,  0.00525446]),
 array([ 0.00513007, -0.01696407,  0.24819538, -0.57975022]),
 array([-0.00703489,  0.01191812, -0.85259653,  1.9914285 ]),
 array([-0.05416082,  0.1202436 , -1.50181414,  3.41992948]),
 array([-0.13540815,  0.30454829, -2.54870201,  5.75458487])]

In [51]:
len(env._action)

4

In [3]:
state_history = env._state.copy()
action_history = env._action.copy()

In [52]:
len(env._state)

6

In [45]:
def run_task(*_):
    # Please note that different environments with different action spaces may
    # require different policies. For example with a Discrete action space, a
    # CategoricalMLPPolicy works, but for a Box action space may need to use
    # a GaussianMLPPolicy (see the trpo_gym_pendulum.py example)
    env = normalize(GymEnv("InvertedPendulum-v1",record_video=False))

    policy = GaussianMLPPolicy(
        env_spec=env.spec,
        # The neural network policy should have two hidden layers, each with 32 hidden units.
        hidden_sizes=(32, 32)
    )

    baseline = LinearFeatureBaseline(env_spec=env.spec)

    algo = TRPO(
        env=env,
        policy=policy,
        baseline=baseline,
        batch_size=4000,
        max_path_length=env.horizon,
        n_itr=40,
        discount=0.99,
        step_size=0.01,
#         Uncomment both lines (this and the plot parameter below) to enable plotting
#         plot=True,
    )
    algo.train()


run_experiment_lite(
    run_task,
    # Number of parallel workers for sampling
    n_parallel=1,
    # Only keep the snapshot parameters for the last iteration
    snapshot_mode="last",
    # Specifies the seed for the experiment. If this is not provided, a random seed
    # will be used
    seed=1,
    exp_prefix = "trpo_ip_64_64_4000_100"
#     plot=True,
)


python /home/zhusj/repos/rllab/scripts/run_experiment_lite.py  --use_cloudpickle 'True'  --exp_name 'trpo_ip_64_64_4000_100_2017_09_05_15_44_27_0007'  --n_parallel '1'  --log_dir '/home/zhusj/repos/rllab/data/local/trpo-ip-64-64-4000-100/trpo_ip_64_64_4000_100_2017_09_05_15_44_27_0007'  --args_data 'gAJjY2xvdWRwaWNrbGUuY2xvdWRwaWNrbGUKX2ZpbGxfZnVuY3Rpb24KcQAoY2Nsb3VkcGlja2xlLmNsb3VkcGlja2xlCl9tYWtlX3NrZWxfZnVuYwpxAWNjbG91ZHBpY2tsZS5jbG91ZHBpY2tsZQpfYnVpbHRpbl90eXBlCnECWAgAAABDb2RlVHlwZXEDhXEEUnEFKEsASwBLBUsRS0djX2NvZGVjcwplbmNvZGUKcQZYkgAAAHQAAHQBAGQBAGQCAGQDAMKDAQHCgwEAfQEAdAIAZAQAfAEAagMAZAUAZBMAwoMAAn0CAHQEAGQEAHwBAGoDAMKDAAF9AwB0BQBkBwB8AQBkCAB8AgBkCQB8AwBkCgBkCwBkDAB8AQBqBgBkDQBkDgBkDwBkEABkEQBkEgDCgwAIfQQAfAQAagcAwoMAAAFkAABTcQdYBgAAAGxhdGluMXEIhnEJUnEKKE5YEwAAAEludmVydGVkUGVuZHVsdW0tdjFxC1gMAAAAcmVjb3JkX3ZpZGVvcQyJWAgAAABlbnZfc3BlY3ENWAwAAABoaWRkZW5fc2l6ZXNxDksgWAMAAABlbnZxD1gGAAAAcG9saWN5cRBYCAAAAGJhc2VsaW5lcRFYCgAAAGJhdGNoX3NpemVxEk2gD1gPAAAAbWF4X3BhdGhfbGVuZ3RocRNYBQAAAG5faXR

In [46]:
datafile =  '../../data/local/trpo-ip-64-64-4000-100/trpo_ip_64_64_4000_100_2017_09_05_15_44_27_0007/progress.csv'
df = pd.read_csv(datafile)
trace1 = go.Scatter(
                    x=df['Iteration'], y=df['AverageReturn'], # Data
                    mode='lines', name='logx' # Additional options
                   )
layout = go.Layout(title='Simple Plot from csv data',
                   plot_bgcolor='rgb(230, 230,230)')

fig = go.Figure(data=[trace1], layout=layout)

# Plot data in the notebook
plotly.offline.iplot(fig)

In [49]:
layout = go.Layout(title='Simple Plot from csv data',
                   plot_bgcolor='rgb(230, 230,230)')

fig = go.Figure(data=[trace1], layout=layout)

# Plot data in the notebook
plotly.offline.iplot(fig)

In [43]:
layout = go.Layout(title='Simple Plot from csv data',
                   plot_bgcolor='rgb(230, 230,230)')

fig = go.Figure(data=[trace1], layout=layout)

# Plot data in the notebook
plotly.offline.iplot(fig)

In [17]:
 env = normalize(GymEnv("InvertedPendulum-v1"))

2017-09-05 11:16:31.172684 EDT | Warning: skipping Gym environment monitoring since snapshot_dir not configured.


[2017-09-05 11:16:31,175] Making new env: InvertedPendulum-v1


2017-09-05 11:16:31.183398 EDT | observation space: Box(4,)
2017-09-05 11:16:31.185015 EDT | action space: Box(1,)


In [16]:
 env = GymEnv("InvertedPendulum-v1")

2017-09-05 11:14:41.628909 EDT | Warning: skipping Gym environment monitoring since snapshot_dir not configured.


[2017-09-05 11:14:41,631] Making new env: InvertedPendulum-v1


2017-09-05 11:14:41.640232 EDT | observation space: Box(4,)
2017-09-05 11:14:41.641825 EDT | action space: Box(1,)


In [23]:
mb = env.wrapped_env.env.env.model.body_mass

In [26]:
mb = np.array(mb)

In [30]:
mb

array([[ 0.       ],
       [ 4.       ],
       [ 4.8953899]])

In [29]:
mb[1,0] = 4

In [31]:
env.wrapped_env.env.env.model.body_mass = mb

In [6]:
env.model.body_mass

array([[ 0.        ],
       [ 9.42477796],
       [ 4.8953899 ]])

In [3]:
env_name = 'InvertedPendulum-v1'
env = gym.make(env_name).env

[2017-09-05 13:08:05,774] Making new env: InvertedPendulum-v1


In [11]:
env.model.body_mass

array([[ 0.        ],
       [ 9.42477796],
       [ 4.8953899 ]])